In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [1]:
import os
import json
import pandas as pd

dir_path = 'autogptq_eval_result/deepseek-moe-16b-base'  # Directory containing the files

def extract_bits(filename):
    """Extract the bits part from the filename."""
    try:
        return filename.split("_w_bit_")[1].split("_pile")[0]
    except IndexError:
        return None

# Dictionary to hold all data, with bits as keys and metric dictionaries as values
all_data = {}

for filename in os.listdir(dir_path):
    if filename.startswith("eval_result_deepseek-moe-16b-base-gptq_w_bit_") and filename.endswith("_pile"):
        bits = extract_bits(filename)
        if bits:  # Ensure bits part was successfully extracted
            file_path = os.path.join(dir_path, filename)
            try:
                with open(file_path, 'r') as file:
                    content = json.load(file)
                    all_data[bits] = content
            except json.JSONDecodeError:
                # If JSON is invalid, skip this file
                continue

# Convert the collected data into a DataFrame
# This approach ensures all keys across files are considered
df = pd.DataFrame.from_dict(all_data, orient='index')

# Optionally, sort the DataFrame by index (bits) if required
# df.sort_index(inplace=True)

df.index.name = 'Bits'

# Save the DataFrame to a CSV file
csv_path = os.path.join(dir_path, 'evaluation_results_all.csv')
df.to_csv(csv_path)

print(f"CSV file has been saved to {csv_path}")


CSV file has been saved to autogptq_eval_result/deepseek-moe-16b-base/evaluation_results_all.csv


In [7]:
import os
import json
import pandas as pd


quantize_model = []
for filename in os.listdir('autogptq_deepseek-ai'):
    quantize_model.append(filename.split('w_bit_')[1])
    

def extract_bits(filename):
    """Extract the bits part from the filename."""
    try:
        return filename.split("_w_bit_")[1].split("_pile")[0]
    except IndexError:
        return None

eval_model = []

for filename in os.listdir('autogptq_eval_result/deepseek-moe-16b-base'):
    if filename.startswith("eval_result_deepseek-moe-16b-base-gptq_w_bit_") and filename.endswith("_pile"):
        bits = extract_bits(filename)
        eval_model.append(bits)
        
list(set(quantize_model) - set(eval_model))



['moe.shared_8.top2_8.other_2+other_block.8',
 'moe.shared_8.top10_4.other_2+other_block.8',
 'moe.shared_8.top45_4.other_2+other_block.8',
 'moe.shared_8.top35_4.other_2+other_block.8',
 'moe.shared_8.top5_4.other_2+other_block.8',
 'moe.shared_8.top30_4.other_2+other_block.8']

In [8]:
import os
import json
import pandas as pd

def extract_bits(filename):
    """Extract the bits part from the filename."""
    try:
        return filename.split("_w_bit_")[1].split("_pile")[0]
    except IndexError:
        return None

eval_bits = []

for filename in os.listdir('autogptq_eval_result/deepseek-moe-16b-base'):
    if filename.startswith("eval_result_deepseek-moe-16b-base-gptq_w_bit_") and filename.endswith("_pile"):
        bits = extract_bits(filename)
        eval_bits.append(bits)
        
eval_bits
len(eval_bits)

['moe.shared_8.top25_4.other_2+other_block.8',
 'moe.shared_8.top1_4.other_2+other_block.4',
 'moe.shared_4.top2_4.other_2+other_block.8',
 'moe.all_mlp.2+other_block.4',
 'moe.shared_4.other.2+other_block.8',
 'moe.shared_8.top1_8.other_2+other_block.4',
 'moe.shared_8.top2_4.other_2+other_block.8',
 'moe.shared_8.top2_8.other_2+other_block.4',
 'moe.all_mlp.4+other_block.8',
 'moe.shared_8.top1_8.other_2+other_block.8',
 'all_8',
 'all_2',
 'moe.shared_8.top2_4.other_2+other_block.4',
 'moe.shared_8.top20_4.other_2+other_block.4',
 'moe.shared_8.top35_4.other_2+other_block.4',
 'moe.shared_2.other.4+other_block_4',
 'all_4',
 'moe.shared_4.top2_4.other_2+other_block.4',
 'moe.shared_4.top2_8.other_2+other_block.8',
 'moe.shared_8.top20_4.other_2+other_block.8',
 'moe.shared_8.top45_4.other_2+other_block.4',
 'moe.shared_8.top30_4.other_2+other_block.4',
 'moe.shared_2.other.4+other_block.8',
 'moe.shared_4.other.2+other_block_4',
 'moe.shared_8.top10_4.other_2+other_block.4',
 'moe.s

31

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from argparse import ArgumentParser
import csv
import torch
from auto_gptq import moe_quantize_config

class parse_args:
    def __init__(self, bits):
        self.bits = bits
model = AutoModelForCausalLM.from_pretrained('deepseek-ai/deepseek-moe-16b-base', torch_dtype=torch.float16, trust_remote_code=True)

log_data = []

def extract_bits(filename):
    """Extract the bits part from the filename."""
    try:
        return filename.split("_w_bit_")[1].split("_pile")[0]
    except IndexError:
        return None

eval_bits = []

for filename in os.listdir('autogptq_eval_result/deepseek-moe-16b-base'):
    if filename.startswith("eval_result_deepseek-moe-16b-base-gptq_w_bit_") and filename.endswith("_pile"):
        bits = extract_bits(filename)
        eval_bits.append(bits)

print(f"len(eval_bits): {len(eval_bits)}")


for bits in eval_bits:
    args = parse_args(bits)
    
    deeepseek_bit = moe_quantize_config(args)
    
    total_bits_moe = 0
    total_params_moe = 0
    total_bits_self_attn = 0
    total_params_self_attn = 0
    total_bits = 0
    total_params = 0
    
    for name, module in model.named_modules():
        if hasattr(module, 'weight'):
            weight = module.weight.data
            num_params = weight.numel()  # Total number of parameters in the module
            
            if name in deeepseek_bit:
                bit = deeepseek_bit[name]
                total_bits += num_params * bit  # Accumulate total bits for all specified modules
                total_params += num_params
                
            if ('experts' in name or 'shared_experts' in name) and name in deeepseek_bit:
                bit = deeepseek_bit[name]
                total_bits_moe += num_params * bit
                total_params_moe += num_params
                
                # print(f'name {name} | bit {bit}')
                # print(f'total_bits_moe {total_bits_moe} | num_params {num_params} | bit {bit}')
            elif 'self_attn' in name and name in deeepseek_bit:
                bit = deeepseek_bit[name]
                total_bits_self_attn += num_params * bit
                total_params_self_attn += num_params
        
    
    average_bit_moe = total_bits_moe / total_params_moe if total_params_moe > 0 else 0
    average_bit_self_attn = total_bits_self_attn / total_params_self_attn if total_params_self_attn > 0 else 0
    average_bit = total_bits / total_params if total_params > 0 else 0
    print(f"Bits: {bits}")
    print(f"MoE Average Bit: {average_bit_moe}")
    print(f"Self-Attention Average Bit: {average_bit_self_attn}")
    print(f"Average Bit: {average_bit}")
    print('=========================')
    
    data = {
        "Bits": bits,
        "MoE Average Bit": average_bit_moe,
        "Self-Attention Average Bit": average_bit_self_attn,
        "Average Bit": average_bit
    }
    
    # Add the data to the list
    log_data.append(data)

fieldnames = ["Bits", "MoE Average Bit", "Self-Attention Average Bit", "Average Bit"]

# Open a CSV file to write the data
save_path = 'autogptq_eval_result/deepseek_bits_data.csv'
with open(save_path, 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()
    
    # Write the log data
    writer.writerows(log_data)

print(f"Log data has been saved to {save_path}.")


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

len(eval_bits): 36
Bits: moe.shared_8.top25_4.other_2+other_block.8
MoE Average Bit: 2.9393939393939394
Self-Attention Average Bit: 8.0
Average Bit: 3.10974537626634
Bits: moe.shared_8.top1_4.other_2+other_block.4
MoE Average Bit: 2.212121212121212
Self-Attention Average Bit: 4.0
Average Bit: 2.272305252692899
Bits: moe.shared_4.top2_4.other_2+other_block.8
MoE Average Bit: 2.121212121212121
Self-Attention Average Bit: 8.0
Average Bit: 2.3191054071597006
Bits: moe.all_mlp.2+other_block.4
MoE Average Bit: 2.0
Self-Attention Average Bit: 4.0
Average Bit: 2.0673245199615478
Bits: moe.shared_4.other.2+other_block.8
MoE Average Bit: 2.0606060606060606
Self-Attention Average Bit: 8.0
Average Bit: 2.2605394835221717
Bits: moe.shared_8.top1_8.other_2+other_block.4
MoE Average Bit: 2.272727272727273
Self-Attention Average Bit: 4.0
Average Bit: 2.3308711763304273
Bits: moe.shared_8.top2_4.other_2+other_block.8
MoE Average Bit: 2.242424242424242
Self-Attention Average Bit: 8.0
Average Bit: 2.4362

In [10]:
import pandas as pd

df1 = pd.read_csv('/home/LeiFeng/xiaolong/moe_quantize/autogptq_eval_result/deepseek-moe-16b-base/deepseek_bits_data.csv')
df2 = pd.read_csv('/home/LeiFeng/xiaolong/moe_quantize/autogptq_eval_result/deepseek-moe-16b-base/evaluation_results_all.csv')

merged_df = pd.merge(df1, df2, on='Bits', how='outer')

merged_df.to_csv('/home/LeiFeng/xiaolong/moe_quantize/autogptq_eval_result/deepseek-moe-16b-base/evaluation_results_final.csv', index=False)


In [2]:
import pandas as pd

csv1 = pd.read_csv('autogptq_eval_result/deepseek-moe-16b-base/evaluation_results_final.csv')
csv2 = pd.read_csv('autogptq_eval_result/deepseek-moe-16b-base/evaluation_results_final1.csv')

unique_in_csv2 = csv2.merge(csv1, indicator=True, how='left', on=csv2.columns.tolist()).loc[lambda x : x['_merge']=='left_only']

unique_in_csv2 = unique_in_csv2.drop('_merge', axis=1)

unique_in_csv2.to_csv('autogptq_eval_result/deepseek-moe-16b-base/final.csv', index=False)


In [6]:
import re

def parse_config_string(config_string):
    # Initialize the config dictionary with default values
    config_dict = {
        "moe.shared_experts": 0,
        "moe.experts": 0,
        "moe.experts.top_index": 0,
        "moe.experts.top": 0,
        "attention": 0,
    }

    # Extract the shared_experts value
    shared_experts_match = re.search(r"shared_(\d+)", config_string)
    if shared_experts_match:
        config_dict["moe.shared_experts"] = int(shared_experts_match.group(1))

    # Extract the experts value
    experts_match = re.search(r"other_(\d+)", config_string)
    if experts_match:
        config_dict["moe.experts"] = int(experts_match.group(1))

    # Extract the top_index and top values
    top_index_match = re.search(r"top(\d+)_", config_string)
    top_match = re.search(r"top\d+_(\d+)", config_string)
    if top_index_match and top_match:
        config_dict["moe.experts.top_index"] = int(top_index_match.group(1))
        config_dict["moe.experts.top"] = int(top_match.group(1))

    # Extract the attention value
    attention_match = re.search(r"other_block\.(\d+)", config_string)
    if attention_match:
        config_dict["attention"] = int(attention_match.group(1))

    return config_dict

dirs = ['autogptq_eval_result/deepseek-moe-16b-base/evaluation_results_final.csv', 'autogptq_eval_result/deepseek-moe-16b-base/evaluation_results_final1.csv']
for dir in dirs:
    df = pd.read_csv(dir)

    # Initialize columns in the dataframe if they don't exist
    for column in ["moe.shared_experts", "moe.experts", "moe.experts.top_index", "moe.experts.top", "attention"]:
        if column not in df.columns:
            df[column] = 0

    # Process each row's 'Bits' column
    for index, row in df.iterrows():
        config_string = row['Bits']
        config_dict = parse_config_string(config_string)
        for key, value in config_dict.items():
            df.at[index, key] = value
    df.to_csv(dir)